## Deezer playlist dataset et recomandandation de morceaux avec word2vec
The errors that you may see are just keboard interrupts

### Préparation des données
Les données sont sous la forme d'une liste de playlist. Chaque playlist est elle me une liste avec l'identifiant deezer du morçeau suivi de l'identifiant de l'artiste.

In [4]:
# chargement des données de playlist
import numpy as np
data = np.load("./music_2.npy",allow_pickle=True)
[len(data), np.mean([len(p) for p in data])]

[100000, 24.21338]

Le jeu de données sur lequel nous allons trvailler contient 100000 playlist qui sont composeer d'en moyenne 24.1 morceaux. 

In [6]:
# separation des ids de morçeau et d'artist
playlist_track = [list(filter(lambda w: w.split("_")[0]==u"track",playlist)) for playlist in data]
playlist_artist = [list(filter(lambda w: w.split("_")[0]==u"artist",playlist)) for playlist in data]

In [7]:
# nombre de morceaux != ?
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

### Création d'un dictionnnaire de morceau:

In [1]:
# nombre d'occurence de chaque morceaux
track_counts = dict((tracks[i],0) for i in range(0, Vt))
for p in playlist_track:
    for a in p:
        track_counts[a]=track_counts[a]+1;


NameError: name 'Vt' is not defined

In [9]:
# Filtrage des morceaux peu fréquent pour gangner un peu de temps au vue de nos ressource en temps de calcul  
playlist_track_filter = [list(filter(lambda a : track_counts[a]> 1, playlist)) for playlist in playlist_track]
# recupération des comptage
counts  =  np.array(list(track_counts.values()))
# trie
order = np.argsort(-counts)
# création de notre liste d'identifiant deezer
tracks_list_ordered = np.array(list(track_counts.keys()))[order]
# Taille de notre vocabulaire = nombre de morçeau conservés
Vt=np.where(counts[order]==1)[0][0]
# construction d'un dict id_morceaux id [0,Vt]
track_dict = dict((tracks_list_ordered[i],i) for i in range(0, Vt))
# conversion des playlist en liste d'entier
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]

### Création des ensembles d'apprentissage de test et de validation 



In [10]:
# ensemble de test et d'apprentissage
index_tst = np.random.choice(100000,20000)
index_val = np.setdiff1d(range(100000),index_tst)
# le debut de chaque playlist est conservé pour l'apprentissage
play_app  = [corpus_num_track[i][:(len(corpus_num_track[i])-1)] 
             for i in range(len(corpus_num_track)) if len(corpus_num_track[i])>1]
# les deux derniers élemnts pour le test et la validation
play_tst  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_tst if len(corpus_num_track[i])>3])
play_val  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_val if len(corpus_num_track[i])>3])[:10000]


In [11]:
# import de Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,Flatten
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams

### hyper-paramètres de word2vec :

La méthode word2vec fait intervennir un certains nombre d'hyper paramètres.

In [12]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

### Création des tables de probabilité de tirage (lissée) et non lissée

In [13]:
# recupération des comptage
counts = np.array(list(track_counts.values()),dtype='float')[order[:Vt]]
# normalisation
st =  counts/np.sum(counts)
# lissage
st_smooth = np.power(st,samp_coef)
st_smooth = st_smooth/np.sum(st_smooth)

### Construction du réseau word2vec

In [14]:
# entrée deux entier (couple de morceaux)
input_target = Input((1,), dtype='int32')
input_context = Input((1,), dtype='int32')


embedding = Embedding(input_dim = Vt, output_dim = vector_dim, name='embedding')

word_embedding = embedding(input_target)


context_embedding = embedding(input_context)


dot_product = Dot(normalize = True, axes = 2)([word_embedding, context_embedding])
dot_product = Flatten()(dot_product)
output = Dense(1, activation='sigmoid',name="classif")(dot_product)

# definition du modèle
Track2Vec = Model(inputs=[input_target, input_context], outputs=output)
Track2Vec.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

In [15]:
Track2Vec.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        3697230     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dot (Dot)                       (None, 1, 1)         0           embedding[0][0]              

### Création du générateur de données

In [16]:
# fonction générant les données associé a une séquence
def word2vecSampling(seq,window=window_width,neg_samples=neg_sample,neg_sampling_table=st,sub_sampling_table=st_smooth,sub_t=sub_samp):
    # taille du vocabulaire
    V = len(neg_sampling_table)
    # créations des paires positives a partir de la séquence
    positives = skipgrams(sequence=seq, vocabulary_size=V, window_size=window,negative_samples=0)
    ppairs    = np.array(positives[0])
    # sous échantillonage
    if (ppairs.shape[0]>0):
        f = sub_sampling_table[ppairs[:,0]]
        subprob = ((f-sub_t)/f)-np.sqrt(sub_t/f)
        tokeep = (subprob<np.random.uniform(size=subprob.shape[0])) | (subprob<0)
        ppairs = ppairs[tokeep,:]
    nbneg     = ppairs.shape[0]*neg_samples
    # tirage des paires négatives
    if (nbneg > 0):
        negex     = np.random.choice(V, nbneg, p=neg_sampling_table)
        negexcontext = np.repeat(ppairs[:,0],neg_samples)
        npairs    = np.transpose(np.stack([negexcontext,negex]))
        pairs     = np.concatenate([ppairs,npairs],axis=0)
        labels    = np.concatenate([np.repeat(1,ppairs.shape[0]),np.repeat(0,nbneg)])
        perm      = np.random.permutation(len(labels))
        res = [pairs[perm,:],labels[perm]]
    else:
        res=[[],[]]
    return res

In [17]:
import random

def track_ns_generator(corpus_num,nbm):
    while 1:
        # tirage de nbm playlist dans corpus_num
        
        playlists = random.sample(corpus_num, nbm)
        
        # création des données x et y 
        a, b = [],[]
        tracks = np.array([track for playlist in playlists for track in playlist])
        c,y= word2vecSampling(tracks, window_width, neg_sample ,st, st_smooth, sub_samp)
        for i in c:
          a.append(np.array(i[0]))
          b.append(np.array(i[1]))

        x= [np.array(a),np.array(b)]

        yield (x,y)

## Apprentissage 

In [18]:
hist=Track2Vec.fit(track_ns_generator(play_app,min_batch_size),steps_per_epoch = 200,epochs=60)

Epoch 1/60
 42/200 [=====>........................] - ETA: 24s - loss: 0.6894 - accuracy: 0.5461

KeyboardInterrupt: 

## Sauvegarde de l'espace latent

In [19]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = Track2Vec.get_weights()[0]
with open('latent_positions.npy', 'wb') as f:
    np.save(f, vectors_tracks)

## Recharger l'espace :

In [20]:
vectors_tracks=np.load("latent_positions.npy")

## Utilisation en complétion et évaluation

In [21]:
from sklearn.neighbors import KDTree
kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')

In [24]:
def predict_batch(seeds,S,X,kdt):
  c=[]
  for j in seeds:
     _, ind = kdt.query([X[j]],k=S) 
     c.append(ind) 

  return c

In [25]:
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)

KeyboardInterrupt: 

In [26]:
def NDCGatK(val,sug):
    t=0
    for i in range(len(sug)):
        if val[i] in sug[i]:
            h=list(sug[i]).index(val[i])+1
            t+=1/np.log2(h+1)
    return t/len(sug)

In [27]:
def HitatK(playlist, sug):
    t=0
    for h in range(len(sug)): 
        if playlist[h] in sug[h]:
            t=+1
            
    return (t/len(sug))

## Tunning des hyper paramètres



## Application

In [29]:
import pandas as pd
tr_meta=pd.read_csv("./tracks_proj.csv")
joindf = pd.DataFrame({"id":tracks_list_ordered[:Vt],"index":range(Vt)})
meta = tr_meta.merge(joindf, left_on="id",right_on="id")
meta.set_index("index",inplace=True)
meta[["title","artist_name","preview","id"]]

,title,artist_name,preview,id
index,,,,
14086,Alone,Petit Biscuit,http://cdn-preview-8.deezer.com/stream/c-89176...,track_100001884
9768,It Was Always You,Maroon 5,http://cdn-preview-e.deezer.com/stream/c-e24ca...,track_100004586
11888,Unkiss Me,Maroon 5,http://cdn-preview-4.deezer.com/stream/c-42340...,track_100004588
321,Sugar,Maroon 5,http://cdn-preview-b.deezer.com/stream/c-b3342...,track_100004590
12477,Leaving California,Maroon 5,http://cdn-preview-5.deezer.com/stream/c-53dbb...,track_100004592
...,...,...,...,...
5338,Hometown,Twenty One Pilots,http://cdn-preview-2.deezer.com/stream/c-2d107...,track_99976972
9202,Not Today,Twenty One Pilots,http://cdn-preview-9.deezer.com/stream/c-9d2b0...,track_99976974
8386,Goner,Twenty One Pilots,http://cdn-preview-2.deezer.com/stream/c-242d7...,track_99976976


In [33]:
def find_track(title):
    return meta.loc[meta["title"]==title,:].index[0]

tr=find_track("Sugar")
tr

321

## Radio

L'api de deeezer permet de récupérer des informations sur les morceaux du dataset a partit de leur id deezer. Parmis ces infos lorsqu'elle est disponnible une url d'écoute d'un extrait gratuit est fournies.

In [48]:
import urllib.request, json 
def gettrackinfo(number):
    track_url =  "https://api.deezer.com/track/{}".format(tracks_list_ordered[number].split("_")[1])
    with urllib.request.urlopen(track_url) as url:
        data = json.loads(url.read().decode())
    return data
track_apidata = gettrackinfo(find_track("Sing For The Moment"))
track_apidata

{'id': 916426,
 'readable': True,
 'title': 'Sing For The Moment',
 'title_short': 'Sing For The Moment',
 'title_version': '',
 'isrc': 'USIR10211061',
 'link': 'https://www.deezer.com/track/916426',
 'share': 'https://www.deezer.com/track/916426?utm_source=deezer&utm_content=track-916426&utm_term=0_1610998631&utm_medium=web',
 'duration': 339,
 'track_position': 12,
 'disk_number': 1,
 'rank': 748466,
 'release_date': '2007-09-13',
 'explicit_lyrics': True,
 'explicit_content_lyrics': 1,
 'explicit_content_cover': 0,
 'preview': 'https://cdns-preview-6.dzcdn.net/stream/c-6a0f240c7bd580e87697146ed3247ab5-8.mp3',
 'bpm': 164.1,
 'gain': -8.9,
 'available_countries': ['AE',
  'AF',
  'AG',
  'AI',
  'AL',
  'AM',
  'AO',
  'AQ',
  'AR',
  'AS',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BQ',
  'BR',
  'BT',
  'BV',
  'BW',
  'BY',
  'CA',
  'CC',
  'CD',
  'CF',
  'CG',
  'CH',
  'CI',
  'CK',
  'CL',
  'CM',
  'CO'

Nous pouvons écouter un extrait :

In [49]:
from IPython.display import display, Audio, clear_output
display(Audio(track_apidata["preview"],autoplay=True))

In [50]:
import time
def start_radio(seed,nb_candidates,duration,nbsteps=20):
    print(meta.loc[seed,"title"])
    display(Audio(meta.loc[seed,"preview"],autoplay=True))
    time.sleep(duration)
    clear_output()
    already_played = [seed]
    for i in range(nbsteps):
        try:
            next_track = random.choice(list(predict_batch([track], nb_candidates, vectors_tracks, kdt)[0]))
            stop = 0
            while (next_track in already_played) & (stop < 20) :
                next_track = random.choice(list(predict_batch([track], nb_candidates, vectors_tracks, kdt)[0]))
                stop += 1
            track = next_track
            track_apidata = gettrackinfo(track)
            print(track_apidata["title"])
            display(Audio(track_apidata["preview"], autoplay = True))
            time.sleep(duration)
            already_played.append(track)
        except:
            print("track not found")
            pass
        clear_output()

In [ ]:
start_radio(find_track("Sugar"),5,duration =50, nbsteps = 2)

In [ ]:
start_radio(find_track("Sing For The Moment"),5,5,10)